In [1]:
import pandas as pd

In [2]:
surveyRaw = pd.read_csv('Copy of Relationship Survey.csv')
pd.options.display.max_columns = None
# display(surveyRaw)
titles = pd.read_csv('data_titles.csv')
surveyRaw.drop(['Respondent ID', 'Collector ID','Start Date','End Date','IP Address','Email Address','First Name','Last Name','Custom Data 1','Unnamed: 11','Unnamed: 13','Unnamed: 18','Unnamed: 22','Unnamed: 26','Unnamed: 30','Unnamed: 34','Unnamed: 38','Unnamed: 41','Unnamed: 44','Unnamed: 47','Unnamed: 50','Indicate if the statement applies to you during the time with your significant other (please check all that apply)','Unnamed: 57','Unnamed: 63','Unnamed: 64','Unnamed: 66','Unnamed: 76','Unnamed: 79'], axis = 1, inplace=True)

In [3]:
titles

,Education/Knowledge/Street Smarts,partner_education_score,difference_education_score,Financial Choices,partner_financial,difference_financial,Confidence/Self-Esteem,partner_confidence,difference_confidence,Religious/Spiritual Values,partner_religious,difference_religious,Materialism/Superficiality,partner_materialism,difference_materialism,Image/Fashion Sense/Body Modification,partner_image,differnece_image,Occupation/Work Ethic/Self-Discipline,partner_occupation,difference_occupation,"Household Care, Maintenance and Cleanliness",partner_house,difference_house,Communication Style/Manners,partner_communication,difference_communication,Artsy/Creative/Musical,partner_artsy,difference_artsy,Charitable/Philanthropic,partner_charity,difference_charity,Pursuing a Greater Purpose,partner_purpose,difference_purpose,Social Status/Sociability,partner_social,difference_social,"Cultured/Well-traveled/""Woke""",partner_culture,difference_culture,Self-Care/Personal Hygiene/Cleanliness,partner_care,difference_care,Honesty/Dependable/Reliable,partner_honesty,difference_honesty,Family Values,partner_family,difference_family
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

surveyRaw = surveyRaw.dropna(subset=surveyRaw.columns[6:19])
surveyRaw

,How old are you?,What is your gender?,What is your sexual orientation?,What is your relationship status?,How long were you/have you been together?,"How you would describe this relationship? Please answer as honestly as you can. (To be out of someone's league means to be 'better' than them. E.g. ""He/She was out of my league"" = He/She was 'better' than me. E.g. ""I was out of his/her league"" = I was 'better' than him/her)",Which value is/was most important to you in your current relationship or at the end of your relationship. (A value is a lifestyle choice or learned behavior that you respect/admire in a person.),How would you rate yourself for this value in your current relationship or at the end of your relationship.,How would you rate your significant other for this value in your current relationship or at the end of your relationship.,Which value is/was 2nd most important to you in your current relationship or at the end of your relationship.,How would you rate yourself for this value in your current relationship or at the end of your relationship.,How would you rate your significant other for this value in your current relationship or at the end of your relationship.,Which value is/was 3rd most important to you in your current relationship or at the end of your relationship.,How would you rate yourself for this value in your current relationship or at the end of your relationship..1,How would you rate your significant other for this value in your current relationship or at the end of your relationship..1,Which value is/was 4th most important to you in your current relationship or at the end of your relationship.,How would you rate yourself for this value in your current relationship or at the end of your relationship..2,How would you rate your significant other for this value in your current relationship or at the end of your relationship..2,Which value is/was 5th most important to you in your current relationship or at the end of your relationship.,How would you rate yourself for this value in your current relationship or at the end of your relationship..3,How would you rate your significant other for this value in your current relationship or at the end of your relationship..3,"Which trait are/were you MOST drawn to as your ""type"" in your current relationship or at the end of your relationship?(A type is an inherent personality or physical trait that you are drawn to and/or extremely attracted to.)",How would you rate your significant other for this trait in your current relationship or at the end of your relationship.,"Which trait are/were you 2nd most drawn to as your ""type"" in your current relationship or at the end of your relationship?",How would you rate your significant other for this trait in your current relationship or at the end of your relationship..1,"Which trait are/were you 3rd most drawn to as your ""type"" in your current relationship or at the end of your relationship?",How would you rate your significant other for this trait in your current relationship or at the end of your relationship..2,"Which trait are/were you 4th most drawn to as your ""type"" in your current relationship or at the end of your relationship?",How would you rate your significant other for this trait in your current relationship or at the end of your relationship..3,"Which trait are/were you 5th most drawn to as your ""type"" in your current relationship or at the end of your relationship?",How would you rate your significant other for this trait in your current relationship or at the end of your relationship..4,Please indicate your decision-making process when you entered this relationship/situationship,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 65,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 77,Unnamed: 78,Unnamed: 80,Unnamed: 81,Unnamed: 82,Please place the slider where it most accurately 

In [5]:




# iterate thru dataframe
for i, j in surveyRaw.iterrows():
    if i == 0:
        continue
    # check 
    print(i)

    ls = [0 for x in range(len(titles.columns))]

    for k in range(6,19,3):
        if j[k] in titles.columns:
            ls[titles.columns.get_loc(j[k])] = int(j[k+1])
            ls[titles.columns.get_loc(j[k]) + 1] = int(j[k+2])
            ls[titles.columns.get_loc(j[k]) + 2] = int(j[k+1]) - int(j[k+2])
        
    titles.loc[i-1]=ls

titles

1
2
3
4
5
6
7
8
9
10
11
12
13
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
66
67
68
69
70
71
72
74
75
76
77
78
79
80
81


,Education/Knowledge/Street Smarts,partner_education_score,difference_education_score,Financial Choices,partner_financial,difference_financial,Confidence/Self-Esteem,partner_confidence,difference_confidence,Religious/Spiritual Values,partner_religious,difference_religious,Materialism/Superficiality,partner_materialism,difference_materialism,Image/Fashion Sense/Body Modification,partner_image,differnece_image,Occupation/Work Ethic/Self-Discipline,partner_occupation,difference_occupation,"Household Care, Maintenance and Cleanliness",partner_house,difference_house,Communication Style/Manners,partner_communication,difference_communication,Artsy/Creative/Musical,partner_artsy,difference_artsy,Charitable/Philanthropic,partner_charity,difference_charity,Pursuing a Greater Purpose,partner_purpose,difference_purpose,Social Status/Sociability,partner_social,difference_social,"Cultured/Well-traveled/""Woke""",partner_culture,difference_culture,Self-Care/Personal Hygiene/Cleanliness,partner_care,difference_care,Honesty/Dependable/Reliable,partner_honesty,difference_honesty,Family Values,partner_family,difference_family
0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,9.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,7.0,4.0,3.0,8.0,5.0,3.0
1,5.0,6.0,-1.0,0.0,0.0,0.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,8.0,-2.0,4.0,7.0,-3.0
2,8.0,8.0,0.0,9.0,10.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,10.0,-1.0,10.0,10.0,0.0
3,8.0,5.0,3.0,6.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,7.0,8.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.0,8.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,9.0,0.0,8.0,9.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,9.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,8.0,-2.0,9.0,8.0,1.0,0.0,0.0,0.0,10.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,5.0,4.0,0.0,0.0,0.0
77,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,4.0,6.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,2.0,0.0,0.0,0.0,7.0,5.0,2.0,0.0,0.0,0.0
78,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,9.0,-2.0,10.0,10.0,0.0,9.0,10.0,-1.0,0.0,0.0,0.0
79,0.0,0.0,0.0,9.0,9.0,0.0,9.0,9.0,0.0,6.0,9.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0


In [10]:
successful = surveyRaw.loc[['Unnamed: 54':'Unnamed: 82']].fillna(0)
successful

SyntaxError: invalid syntax (<ipython-input-10-ee2435d3f2d3>, line 1)

# Machinge learning using multiple linear regression sklearn

In [7]:

# Generate a linear dataset with n features

from sklearn.datasets import make_regression

n_features = 3
X, y = make_regression(n_samples=30, n_features=n_features, 
                       n_informative=n_features, random_state=42, 
                       noise=0.5, bias=100.0)
print(X.shape)


(30, 3)


In [8]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all of our features in X
model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")

R2 Score: 0.9999838079561164
